In [0]:
%run "./33_setup_data_sets_to_perform_joins"

+---------+--------------------+-------------------+---------+-------------------+
|course_id|        course_title|course_published_dt|is_active|    last_updated_ts|
+---------+--------------------+-------------------+---------+-------------------+
|        1|    Mastering Python|         2021-01-14|     true|2021-02-18 16:57:25|
|        2|Data Engineering ...|         2021-02-10|     true|2021-03-05 12:07:33|
|        3|   Mastering Pyspark|         2021-01-07|     true|2021-04-06 10:05:42|
|        4|      AWS Essentials|         2021-03-19|    false|2021-04-10 02:25:36|
|        5|          Docker 101|         2021-02-28|     true|2021-03-21 07:18:52|
+---------+--------------------+-------------------+---------+-------------------+



+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+



+-------------------+-------+---------+----------+
|course_enrolment_id|user_id|course_id|price_paid|
+-------------------+-------+---------+----------+
|                  1|     10|        2|      9.99|
|                  2|      5|        2|      9.99|
|                  3|      7|        5|     10.99|
|                  4|      9|        2|      9.99|
|                  5|      8|        2|      9.99|
|                  6|      5|        5|     10.99|
|                  7|      4|        5|     10.99|
|                  8|      7|        3|     10.99|
|                  9|      8|        5|     10.99|
|                 10|      3|        3|     10.99|
|                 11|      7|        5|     10.99|
|                 12|      3|        2|      9.99|
|                 13|      5|        2|      9.99|
|                 14|      4|        3|     10.99|
|                 15|      8|        2|      9.99|
+-------------------+-------+---------+----------+



In [0]:
help(courses_df.join)

Help on method join in module pyspark.sql.dataframe:

join(other: 'DataFrame', on: Union[str, List[str], pyspark.sql.column.Column, List[pyspark.sql.column.Column], NoneType] = None, how: Optional[str] = None) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Joins with another :class:`DataFrame`, using the given join expression.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    other : :class:`DataFrame`
        Right side of the join
    on : str, list or :class:`Column`, optional
        a string for the join column name, a list of column names,
        a join expression (Column), or a list of Columns.
        If `on` is a string or a list of strings indicating the name of the join column(s),
        the column(s) must exist on both sides, and this performs an equi-join.
    how : str, optional
        default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
      

In [0]:
users_df.join(course_enrolments_df, on='user_id', how='inner').show()

+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|        3|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  2|        2|      9.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  6|        5|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|            

In [0]:
# the column on which the dataframes are joined get repeated in the result with this syntax

users_df.alias("u").join(
    course_enrolments_df.alias("ce"), users_df["user_id"] == course_enrolments_df["user_id"], how="inner"
).show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|      3|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|      3|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|      4|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|      4|        3|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  2|      5|        2|      9.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  6|      5|        5|     10.99|
|

### Join example

* Find the total number of courses enrolled by each user

In [0]:
from pyspark.sql import functions as F

result_df = (
    users_df.alias("u")
    .join(course_enrolments_df.alias("ce"), on=["user_id"], how="left")
    .groupBy("u.user_id")
    .agg(
        F.sum(F.when(F.col("course_enrolment_id").isNull(), 0).otherwise(1)).alias(
            "courses_enrolled"
        )
    )
    .orderBy(F.desc("courses_enrolled"))
)

result_df.show()

+-------+----------------+
|user_id|courses_enrolled|
+-------+----------------+
|      5|               3|
|      7|               3|
|      8|               3|
|      3|               2|
|      4|               2|
|      9|               1|
|     10|               1|
|      1|               0|
|      2|               0|
|      6|               0|
+-------+----------------+



### Another way of writing the same query

In [0]:
from pyspark.sql import functions as F

result_df = (
    users_df.alias("u")
    .join(course_enrolments_df.alias("ce"), on=["user_id"], how="left")
    .groupBy("u.user_id")
    .agg(
        F.sum(
            F.expr(
                """
                    CASE 
                        WHEN 
                            ce.course_enrolment_id is NULL 
                        THEN 0 
                        ELSE 1 
                    END
                """
            )
        ).alias("courses_enrolled")
    )
    .orderBy(F.desc("courses_enrolled"))
)

result_df.show()

+-------+----------------+
|user_id|courses_enrolled|
+-------+----------------+
|      5|               3|
|      7|               3|
|      8|               3|
|      3|               2|
|      4|               2|
|      9|               1|
|     10|               1|
|      1|               0|
|      2|               0|
|      6|               0|
+-------+----------------+

